# Long-term Memory: Cross-Session Knowledge

## Introduction

In this notebook, you'll learn about long-term memory - persistent knowledge that survives across sessions. While working memory handles the current conversation, long-term memory stores important facts, preferences, and experiences that should be remembered indefinitely.

### What You'll Learn

- What long-term memory is and why it's essential
- The three types of long-term memories: semantic, episodic, and message
- How to store and retrieve long-term memories
- How semantic search works with memories
- How automatic deduplication prevents redundancy

### Prerequisites

- Completed Section 2 notebooks
- Completed `01_working_memory_with_extraction_strategies.ipynb`
- Redis 8 running locally
- Agent Memory Server running
- OpenAI API key set

## Concepts: Long-term Memory

### What is Long-term Memory?

Long-term memory is **persistent, cross-session knowledge** about users, preferences, and important facts. Unlike working memory (which is session-scoped), long-term memory:

- ✅ Survives across sessions
- ✅ Accessible from any conversation
- ✅ Searchable via semantic vector search
- ✅ Automatically deduplicated
- ✅ Organized by user/namespace

### Working Memory vs. Long-term Memory

| Working Memory | Long-term Memory |
|----------------|------------------|
| **Session-scoped** | **User-scoped** |
| Current conversation | Important facts |
| TTL-based (expires) | Persistent |
| Full message history | Extracted knowledge |
| Loaded/saved each turn | Searched when needed |

### Three Types of Long-term Memories

The Agent Memory Server supports three types of long-term memories:

1. **Semantic Memory** - Facts and knowledge
   - Example: "Student prefers online courses"
   - Example: "Student's major is Computer Science"
   - Example: "Student wants to graduate in 2026"

2. **Episodic Memory** - Events and experiences
   - Example: "Student enrolled in CS101 on 2024-09-15"
   - Example: "Student asked about machine learning on 2024-09-20"
   - Example: "Student completed Data Structures course"

3. **Message Memory** - Important conversation snippets
   - Example: Full conversation about career goals
   - Example: Detailed discussion about course preferences

### How Semantic Search Works

Long-term memories are stored with vector embeddings, enabling semantic search:

- Query: "What does the student like?"
- Finds: "Student prefers online courses", "Student enjoys programming"
- Even though exact words don't match!

### Automatic Deduplication

The Agent Memory Server automatically prevents duplicate memories:

- **Hash-based**: Exact duplicates are rejected
- **Semantic**: Similar memories are merged
- Keeps memory storage efficient

## Setup

In [ ]:
import os
import asyncio
from datetime import datetime
from redis_context_course import MemoryClient

# Initialize memory client
student_id = "student_123"
memory_client = MemoryClient(
    user_id=student_id,
    namespace="redis_university"
)

print(f"✅ Memory client initialized for {student_id}")

## Hands-on: Working with Long-term Memory

### Example 1: Storing Semantic Memories (Facts)

Let's store some facts about the student.

In [ ]:
# Store student preferences
await memory_client.create_memory(
    text="Student prefers online courses over in-person classes",
    memory_type="semantic",
    topics=["preferences", "course_format"]
)

await memory_client.create_memory(
    text="Student's major is Computer Science with a focus on AI/ML",
    memory_type="semantic",
    topics=["academic_info", "major"]
)

await memory_client.create_memory(
    text="Student wants to graduate in Spring 2026",
    memory_type="semantic",
    topics=["goals", "graduation"]
)

await memory_client.create_memory(
    text="Student prefers morning classes, no classes on Fridays",
    memory_type="semantic",
    topics=["preferences", "schedule"]
)

print("✅ Stored 4 semantic memories (facts about the student)")

### Example 2: Storing Episodic Memories (Events)

Let's store some events and experiences.

In [ ]:
# Store course enrollment events
await memory_client.create_memory(
    text="Student enrolled in CS101: Introduction to Programming on 2024-09-01",
    memory_type="episodic",
    topics=["enrollment", "courses"],
    metadata={"course_code": "CS101", "date": "2024-09-01"}
)

await memory_client.create_memory(
    text="Student completed CS101 with grade A on 2024-12-15",
    memory_type="episodic",
    topics=["completion", "grades"],
    metadata={"course_code": "CS101", "grade": "A", "date": "2024-12-15"}
)

await memory_client.create_memory(
    text="Student asked about machine learning courses on 2024-09-20",
    memory_type="episodic",
    topics=["inquiry", "machine_learning"],
    metadata={"date": "2024-09-20"}
)

print("✅ Stored 3 episodic memories (events and experiences)")

### Example 3: Searching Memories with Semantic Search

Now let's search for memories using natural language queries.

In [ ]:
# Search for preferences
print("Query: 'What does the student prefer?'\n")
results = await memory_client.search_memories(
    query="What does the student prefer?",
    limit=3
)

for i, memory in enumerate(results, 1):
    print(f"{i}. {memory.text}")
    print(f"   Type: {memory.memory_type} | Topics: {', '.join(memory.topics)}")
    print()

In [ ]:
# Search for academic information
print("Query: 'What is the student studying?'\n")
results = await memory_client.search_memories(
    query="What is the student studying?",
    limit=3
)

for i, memory in enumerate(results, 1):
    print(f"{i}. {memory.text}")
    print(f"   Type: {memory.memory_type}")
    print()

In [ ]:
# Search for course history
print("Query: 'What courses has the student taken?'\n")
results = await memory_client.search_memories(
    query="What courses has the student taken?",
    limit=3
)

for i, memory in enumerate(results, 1):
    print(f"{i}. {memory.text}")
    print(f"   Type: {memory.memory_type}")
    if memory.metadata:
        print(f"   Metadata: {memory.metadata}")
    print()

### Example 4: Demonstrating Deduplication

Let's try to store duplicate memories and see how deduplication works.

In [ ]:
# Try to store an exact duplicate
print("Attempting to store exact duplicate...")
try:
    await memory_client.create_memory(
        text="Student prefers online courses over in-person classes",
        memory_type="semantic",
        topics=["preferences", "course_format"]
    )
    print("❌ Duplicate was stored (unexpected)")
except Exception as e:
    print(f"✅ Duplicate rejected: {e}")

# Try to store a semantically similar memory
print("\nAttempting to store semantically similar memory...")
try:
    await memory_client.create_memory(
        text="Student likes taking classes online instead of on campus",
        memory_type="semantic",
        topics=["preferences", "course_format"]
    )
    print("Memory stored (may be merged with existing similar memory)")
except Exception as e:
    print(f"✅ Similar memory rejected: {e}")

### Example 5: Cross-Session Memory Access

Let's simulate a new session and show that memories persist.

In [ ]:
# Create a new memory client (simulating a new session)
new_session_client = MemoryClient(
    user_id=student_id,  # Same user
    namespace="redis_university"
)

print("New session started for the same student\n")

# Search for memories from the new session
print("Query: 'What do I prefer?'\n")
results = await new_session_client.search_memories(
    query="What do I prefer?",
    limit=3
)

print("✅ Memories accessible from new session:\n")
for i, memory in enumerate(results, 1):
    print(f"{i}. {memory.text}")
    print()

### Example 6: Filtering by Memory Type and Topics

In [ ]:
# Get all semantic memories
print("All semantic memories (facts):\n")
results = await memory_client.search_memories(
    query="",  # Empty query returns all
    memory_type="semantic",
    limit=10
)

for i, memory in enumerate(results, 1):
    print(f"{i}. {memory.text}")
    print(f"   Topics: {', '.join(memory.topics)}")
    print()

In [ ]:
# Get all episodic memories
print("All episodic memories (events):\n")
results = await memory_client.search_memories(
    query="",
    memory_type="episodic",
    limit=10
)

for i, memory in enumerate(results, 1):
    print(f"{i}. {memory.text}")
    if memory.metadata:
        print(f"   Metadata: {memory.metadata}")
    print()

## Key Takeaways

### When to Use Long-term Memory

Store in long-term memory:
- ✅ User preferences and settings
- ✅ Important facts about the user
- ✅ Goals and objectives
- ✅ Significant events and milestones
- ✅ Completed courses and achievements

Don't store in long-term memory:
- ❌ Temporary conversation context
- ❌ Trivial details
- ❌ Information that changes frequently
- ❌ Sensitive data without proper handling

### Memory Types Guide

**Semantic (Facts):**
- "Student prefers X"
- "Student's major is Y"
- "Student wants to Z"

**Episodic (Events):**
- "Student enrolled in X on DATE"
- "Student completed Y with grade Z"
- "Student asked about X on DATE"

**Message (Conversations):**
- Important conversation snippets
- Detailed discussions worth preserving

### Best Practices

1. **Use descriptive topics** - Makes filtering easier
2. **Add metadata** - Especially for episodic memories
3. **Write clear memory text** - Will be searched semantically
4. **Let deduplication work** - Don't worry about duplicates
5. **Search before storing** - Check if similar memory exists

## Exercises

1. **Store your own memories**: Create 5 semantic and 3 episodic memories about a fictional student. Search for them.

2. **Test semantic search**: Create memories with different wordings but similar meanings. Search with various queries to see what matches.

3. **Explore metadata**: Add rich metadata to episodic memories. How can you use this in your agent?

4. **Cross-session test**: Create a memory, close the notebook, restart, and verify the memory persists.

## Summary

In this notebook, you learned:

- ✅ Long-term memory stores persistent, cross-session knowledge
- ✅ Three types: semantic (facts), episodic (events), message (conversations)
- ✅ Semantic search enables natural language queries
- ✅ Automatic deduplication prevents redundancy
- ✅ Memories are user-scoped and accessible from any session

**Next:** In the next notebook, we'll integrate working memory and long-term memory to build a complete memory system for our agent.